# **ANÁLISIS EXPLORATORIO DE DATOS (EDA) - NFL**
#### *THE BRIDGE - DATA_SCIENCE 2023*

In [78]:
#Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nfl_data_py as nfl
import seaborn as sns
import warnings
import os 
warnings.filterwarnings("ignore")

### ***Hipótesis.***

1.  **¿Cuál es la relación entre lesiones y éxito en la NFL?**
2.  **¿Las consideras lesiones severas son las más habituales?**
3.  **¿Las lesiones leves pueden terminar con la carrera de los profesionales?**    
4.  **¿En qué época de la temporada regular existen más lesiones?**
5.  **¿Cuál es la posición que tiende lesionarse?**
6.  **¿Qué hace la NFL para intentar reducir el número de lesiones para que no acaben con la carrera/vida de los jugadores? *(pregunta a futuro)***


In [2]:
#Datasets

injuries = nfl.import_injuries([2011,2012,2013,2014,2015,2016,2017,2018,2019,2020])
# stats = nfl.import_ngs_data("passing",[2011,2012,2013,2014,2015,2016,2017,2018,2019,2020])
rank_11 = pd.read_csv("/Users/ivanlamb/Desktop/Workspace/eda_nfl/src/data/raw/Rankings/Rank_nfl_2011.csv")
rank_12 = pd.read_csv("/Users/ivanlamb/Desktop/Workspace/eda_nfl/src/data/raw/Rankings/Rank_nfl_2012.csv")
rank_13 = pd.read_csv("/Users/ivanlamb/Desktop/Workspace/eda_nfl/src/data/raw/Rankings/Rank_nfl_2013.csv")
rank_14 = pd.read_csv("/Users/ivanlamb/Desktop/Workspace/eda_nfl/src/data/raw/Rankings/Rank_nfl_2014.csv")
rank_15 = pd.read_csv("/Users/ivanlamb/Desktop/Workspace/eda_nfl/src/data/raw/Rankings/Rank_nfl_2015.csv")
rank_16 = pd.read_csv("/Users/ivanlamb/Desktop/Workspace/eda_nfl/src/data/raw/Rankings/Rank_nfl_2016.csv")
rank_17 = pd.read_csv("/Users/ivanlamb/Desktop/Workspace/eda_nfl/src/data/raw/Rankings/Rank_nfl_2017.csv")
rank_18 = pd.read_csv("/Users/ivanlamb/Desktop/Workspace/eda_nfl/src/data/raw/Rankings/Rank_nfl_2018.csv")
rank_19 = pd.read_csv("/Users/ivanlamb/Desktop/Workspace/eda_nfl/src/data/raw/Rankings/Rank_nfl_2019.csv")
rank_20 = pd.read_csv("/Users/ivanlamb/Desktop/Workspace/eda_nfl/src/data/raw/Rankings/Rank_nfl_2020.csv")

In [3]:
# Asignamos una nueva columna llamada "Year" en cada Dataset

rank_11 = rank_11.assign(Year = 2011.0)
rank_12 = rank_12.assign(Year = 2012.0)
rank_13 = rank_13.assign(Year = 2013.0)
rank_14 = rank_14.assign(Year = 2014.0)
rank_15 = rank_15.assign(Year = 2015.0)
rank_16 = rank_16.assign(Year = 2016.0)
rank_17 = rank_17.assign(Year = 2017.0)
rank_18 = rank_18.assign(Year = 2018.0)
rank_19 = rank_19.assign(Year = 2019.0)
rank_20 = rank_20.assign(Year = 2020.0)

In [5]:
# Creamos una lista para después concatecar todas los Datasets en uno solo

ranks_10_20 = [rank_11, rank_12, rank_13, rank_14, rank_15, rank_16, rank_17, rank_18, rank_19, rank_20,]

list_data = []
  
for i in ranks_10_20:
    data = (i)
    list_data.append(data)
 
ranks_nfl = pd.concat(list_data,ignore_index=True)

In [6]:
#Elimanos todas las columnas inecesarias y con un reindex organizamos nuestro Dataset

ranks_nfl = ranks_nfl.reindex(columns=["Rank", "Player", "Position", "Year"])

In [7]:
#Analisamos nuestro primer Dataset "ranks_nfl"

ranks_nfl.head()

,Rank,Player,Position,Year
0,1,Tom Brady,Quarterback,2011.0
1,2,Peyton Manning,Quarterback,2011.0
2,3,Adrian Peterson,Running back,2011.0
3,4,Ray Lewis,Linebacker,2011.0
4,5,Ed Reed,Safety,2011.0


In [8]:
ranks_nfl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Rank      1000 non-null   int64  
 1   Player    1000 non-null   object 
 2   Position  1000 non-null   object 
 3   Year      1000 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 31.4+ KB


In [9]:
injuries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52289 entries, 0 to 5660
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   season                     52289 non-null  float64       
 1   game_type                  52289 non-null  object        
 2   team                       52289 non-null  object        
 3   week                       52289 non-null  float64       
 4   gsis_id                    52289 non-null  object        
 5   position                   52289 non-null  object        
 6   full_name                  52289 non-null  object        
 7   first_name                 52289 non-null  object        
 8   last_name                  52289 non-null  object        
 9   report_primary_injury      38094 non-null  object        
 10  report_secondary_injury    2177 non-null   object        
 11  report_status              38094 non-null  object        
 12  pract

In [10]:
injuries.drop(columns=["gsis_id", "first_name",
                                  "last_name",
                                  "report_secondary_injury",
                                  "practice_secondary_injury"],
              inplace=True)
injuries.rename(columns={'season':'Year',
                         'game_type':'Season',
                         'team':'Team',
                         'position':'Position',
                         'full_name': 'Player',
                         'week': 'Week'},
                inplace=True)

In [11]:
#Reduciendo el numero datos faltantes

injuries.dropna(inplace=True)
injuries.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38020 entries, 0 to 5660
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Year                     38020 non-null  float64       
 1   Season                   38020 non-null  object        
 2   Team                     38020 non-null  object        
 3   Week                     38020 non-null  float64       
 4   Position                 38020 non-null  object        
 5   Player                   38020 non-null  object        
 6   report_primary_injury    38020 non-null  object        
 7   report_status            38020 non-null  object        
 8   practice_primary_injury  38020 non-null  object        
 9   practice_status          38020 non-null  object        
 10  date_modified            38020 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(8)
memory usage: 3.5+ MB


In [12]:
ranks_nfl["Position"].value_counts()

Wide receiver         165
Quarterback           140
Linebacker            123
Running back          108
Defensive end         106
Cornerback             78
Defensive tackle       61
Safety                 56
Tight end              54
Offensive tackle       46
Guard                  25
Center                 12
Outside linebacker     11
Free safety             5
Fullback                3
Offensive Tackle        3
Kicker                  1
Defensive Tackle        1
Wide Receiver           1
Strong safety           1
Name: Position, dtype: int64

In [26]:
injuries["Position"].value_counts()

Linebacker          5685
Wide receiver       5003
Cornerback          4728
Running back        3440
Safety              3403
Defense end         3007
Tackle              2716
Defensive tackle    2563
Tight end           2508
Guard               2236
Quarterback         1115
Center              1097
Kicker               234
Punter               171
Long snapper          99
Kicker returner       14
Punt Returner          1
Name: Position, dtype: int64

In [22]:
injuries["Position"].replace({"LB":"Linebacker",
                              "WR":"Wide receiver",
                              "CB":"Cornerback",
                              "RB":"Running back",
                              "S":"Safety",
                              "DE":"Defense end",
                              "T":"Tackle",
                              "DT":"Defensive tackle",
                              "TE": "Tight end",
                              "G": "Guard",
                              "QB":"Quarterback",
                              "C": "Center",
                              "K": "Kicker",
                              "P": "Punter",
                              "LS":"Long snapper",
                              "KR": "Kicker returner",
                              "PR":"Punt returner"}, inplace=True)

In [ ]:
ranks_nfl["Position"].replace({"LB":"Linebacker",
                              "WR":"Wide receiver",
                              "CB":"Cornerback",
                              "RB":"Running back",
                              "S":"Safety",
                              "DE":"Defense end",
                              "T":"Tackle",
                              "DT":"Defensive tackle",
                              "TE": "Tight end",
                              "G": "Guard",
                              "QB":"Quarterback",
                              "C": "Center",
                              "K": "Kicker",
                              "P": "Punter",
                              "LS":"Long snapper",
                              "KR": "Kicker returner",
                              "PR":"Punt returner"}, inplace=True)

In [14]:
injuries[injuries["Position"] == "Tight End"]

,Year,Season,Team,Week,Position,Player,report_primary_injury,report_status,practice_primary_injury,practice_status,date_modified


In [15]:
injuries["Position"]
                            

0       TE
1       QB
2       CB
3       TE
4       CB
        ..
5648     T
5649    LB
5650    WR
5659    TE
5660    WR
Name: Position, Length: 38020, dtype: object

In [86]:
#Recuduciendo en DF para quedarme solo con los juegos de temporada regular

injuries = injuries[injuries["Season"].str.contains("REG")]

In [65]:
#Reduciendo el DF para quedarme solo con los jugadores que estan "Out"

# injuries[injuries["practice_status"].str.contains("Did Not Participate In Practice")]
# injuries[injuries["report_status"].str.contains("Out")]


TypeError: 'NoneType' object is not subscriptable

In [128]:
#Calculando con un groupby el numero de veces que aparece un jugador en el DF.
# pd.value_counts(injuries['full_name'].str.contains["Julian Edelman"])
injuries[injuries["full_name"].str.contains("Julian Edelman")].head(30)

,season,game_type,team,week,position,full_name,report_primary_injury,report_status,practice_primary_injury,practice_status,date_modified
135,2011.0,REG,NE,1.0,WR,Julian Edelman,Hand,Probable,Hand,Full Participation in Practice,2011-09-10 15:32:13
1199,2011.0,REG,NE,5.0,WR,Julian Edelman,Ankle,Doubtful,Ankle,Did Not Participate In Practice,2011-10-07 12:32:43
1484,2011.0,REG,NE,6.0,WR,Julian Edelman,Ankle,Questionable,Ankle,Limited Participation in Practice,2011-10-14 12:50:31
3385,2011.0,REG,NE,13.0,WR,Julian Edelman,Back,Questionable,Back,Limited Participation in Practice,2011-12-02 12:54:09
3694,2011.0,REG,NE,14.0,WR,Julian Edelman,Back,Questionable,Back,Limited Participation in Practice,2011-12-09 12:35:41
4021,2011.0,REG,NE,15.0,WR,Julian Edelman,Back,Questionable,Back,Limited Participation in Practice,2011-12-16 12:15:25
4332,2011.0,REG,NE,16.0,WR,Julian Edelman,Back,Questionable,Back,Limited Participation in Practice,2011-12-23 12:07:23
4628,2011.0,REG,NE,17.0,WR,Julian Edelman,Back,Questionable,Back,Limited Participation in Practice,2011-12-30 12:41:14
942,2012.0,REG,NE,4.0,WR,Julian Edelman,Hand,Out,Hand,Did Not Participate In Practice,2012-09-28 13:01:37
1207,2012.0,REG,NE,5.0,WR,Julian Edelman,Hand,Out,Hand,Did Not Participate In Practice,2012-10-05 12:20:12


In [121]:
injuries[injuries["report_primary_injury"]=="Not Injury Related"]

,season,game_type,team,week,position,full_name,report_primary_injury,report_status,practice_primary_injury,practice_status,date_modified
328,2011.0,REG,IND,2.0,DE,Dwight Freeney,Not Injury Related,Probable,Not Injury Related,Full Participation in Practice,2011-09-16 12:59:36
329,2011.0,REG,IND,2.0,WR,Reggie Wayne,Not Injury Related,Probable,Not Injury Related,Full Participation in Practice,2011-09-16 13:04:21
509,2011.0,REG,CIN,3.0,WR,Jerome Simpson,Not Injury Related,Questionable,Not Injury Related,Did Not Participate In Practice,2011-09-23 11:49:34
519,2011.0,REG,CLE,3.0,DE,Marcus Benard,Not Injury Related,Probable,Not Injury Related,Full Participation in Practice,2011-09-23 12:24:13
657,2011.0,REG,NYG,3.0,DE,Justin Tuck,Not Injury Related,Probable,Not Injury Related,Did Not Participate In Practice,2011-09-23 10:26:30
...,...,...,...,...,...,...,...,...,...,...,...
5059,2020.0,REG,ATL,17.0,DE,Charles Harris,Not Injury Related,Questionable,Not Injury Related,Did Not Participate In Practice,2021-01-01 11:17:02
5265,2020.0,REG,MIN,17.0,RB,Dalvin Cook,Not Injury Related,Out,Not Injury Related,Did Not Participate In Practice,2021-01-01 11:27:33
5347,2020.0,REG,PIT,17.0,QB,Ben Roethlisberger,Not Injury Related,Out,Not Injury Related,Did Not Participate In Practice,2021-01-01 13:27:36
5374,2020.0,REG,SF,17.0,DT,Kevin Givens,Not Injury Related,Out,Not Injury Related,Did Not Participate In Practice,2021-01-01 12:32:13


In [ ]:
#Filtro() de todos los repetidos(para comprobar)

In [ ]:
freq_repo_injury = injuries.groupby(["report_primary_injury"]).count()
pd.unique(injuries["report_primary_injury"])

In [47]:
injuries["report_primary_injury"].value_counts()

report_primary_injury
Knee                     6871
Ankle                    5423
Hamstring                4029
Shoulder                 2943
Foot                     1972
                         ... 
Quadriceps                  1
Non-Football Illness        1
Mouth                       1
Lung contusion              1
tricep                      1
Length: 155, dtype: int64

##### ***Fuentes***
- Datasets: injuries: [Libreria NFL]("https://pypi.org/project/nfl-data-py/"), [Wikipedia]("https://en.wikipedia.org/wiki/NFL_Top_100")